# Hola, mundo en LangChain

## Instalar librerías principales y configuración de API Key de OpenAI

In [1]:
%%capture
!pip install langchain pypdf openai chromadb tiktoken

In [2]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

Enter the secret value: ··········


## Carga de documents

In [3]:
import requests
from langchain.document_loaders import PyPDFLoader

urls = [
    'https://arxiv.org/pdf/2306.06031v1.pdf',
    'https://arxiv.org/pdf/2306.12156v1.pdf',
    'https://arxiv.org/pdf/2306.14289v1.pdf',
    'https://arxiv.org/pdf/2305.10973v1.pdf',
    'https://arxiv.org/pdf/2306.13643v1.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

Descargado paper1.pdf
Descargado paper2.pdf
Descargado paper3.pdf
Descargado paper4.pdf
Descargado paper5.pdf
Contenido de ml_papers:



In [4]:
type(ml_papers), len(ml_papers), ml_papers[3]

(list,
 57,
 Document(page_content='Figure 1: FinGPT Framework.\n4.1 Data Sources\nThe first stage of the FinGPT pipeline involves the collec-\ntion of extensive financial data from a wide array of online\nsources. These include, but are not limited to:\n•Financial news: Websites such as Reuters, CNBC, Yahoo\nFinance, among others, are rich sources of financial news\nand market updates. These sites provide valuable informa-\ntion on market trends, company earnings, macroeconomic\nindicators, and other financial events.\n•Social media : Platforms such as Twitter, Facebook, Red-\ndit, Weibo, and others, offer a wealth of information in\nterms of public sentiment, trending topics, and immediate\nreactions to financial news and events.\n•Filings : Websites of financial regulatory authorities, such\nas the SEC in the United States, offer access to company\nfilings. These filings include annual reports, quarterly earn-\nings, insider trading reports, and other important company-\nspecific in

## Split de documents

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
    )

documents = text_splitter.split_documents(ml_papers)

In [6]:
len(documents), documents[10]

(211,
 Document(page_content='highly volatile, changing rapidly in response to news events\nor market movements.\nTrends , often observable through websites like Seeking\nAlpha, Google Trends, and other finance-oriented blogs and\nforums, offer critical insights into market movements and in-\nvestment strategies. They feature:\n•Analyst perspectives: These platforms provide access to\nmarket predictions and investment advice from seasoned\nfinancial analysts and experts.\n•Market sentiment: The discourse on these platforms can\nreflect the collective sentiment about specific securities,\nsectors, or the overall market, providing valuable insights\ninto the prevailing market mood.\n•Broad coverage: Trends data spans diverse securities and\nmarket segments, offering comprehensive market coverage.\nEach of these data sources provides unique insights into\nthe financial world. By integrating these diverse data types,\nfinancial language models like FinGPT can facilitate a com-\nprehensive 

## Embeddings e ingesta a base de datos vectorial

In [8]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
    )

## Modelos de chat y cadenas para consulta de información

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [10]:
query = "qué es fingpt?"
qa_chain.run(query)

'FinGPT es un modelo de lenguaje financiero de código abierto desarrollado por la comunidad AI4Finance. Está diseñado para aplicaciones en el campo de las finanzas y utiliza técnicas de procesamiento de lenguaje natural para analizar y comprender datos financieros. FinGPT adopta un enfoque centrado en los datos y utiliza métodos rigurosos de limpieza y preprocesamiento para garantizar la calidad de los datos. También ofrece un marco de trabajo de extremo a extremo con cuatro capas, que abarca desde la obtención de datos hasta la implementación de aplicaciones prácticas en finanzas. El objetivo de FinGPT es estimular la innovación, democratizar los modelos de lenguaje financiero y desbloquear nuevas oportunidades en el campo de las finanzas abiertas.'

In [11]:
query = "qué hace complicado entrenar un modelo como el fingpt?"
qa_chain.run(query)

'Entrenar un modelo como FinGPT puede ser complicado por varias razones. En primer lugar, los modelos de lenguaje grandes requieren una gran cantidad de recursos computacionales, como unidades de procesamiento gráfico (GPU) y tiempo de entrenamiento. En el caso de BloombergGPT, se utilizaron aproximadamente 1.3 millones de horas de GPU para el entrenamiento, lo que se traduce en un costo significativo.\n\nAdemás, la adquisición de datos financieros de alta calidad y relevantes puede ser un desafío. Obtener datos financieros históricos o especializados de diferentes fuentes, como plataformas web, API, documentos PDF e imágenes, puede ser complejo debido a los diferentes formatos y tipos de datos.\n\nOtro desafío es la naturaleza dinámica del dominio financiero. El mundo financiero está en constante cambio, y los modelos de lenguaje deben actualizarse regularmente para mantener su precisión y relevancia. Esto requiere una capacidad de adaptación rápida y eficiente del modelo, lo cual pue

In [12]:
query = "qué es fast segment?"
qa_chain.run(query)

'Fast Segment es un método alternativo propuesto en el artículo "Fast Segment Anything" para acelerar el modelo Segment Anything (SAM) en tareas de visión por computadora. SAM es un modelo que puede segmentar cualquier objeto en una imagen y ha demostrado resultados prometedores en diversas tareas. Sin embargo, SAM tiene altos costos computacionales debido a su arquitectura Transformer en entradas de alta resolución. Fast Segment propone una forma más rápida de lograr resultados comparables al entrenar un método existente de segmentación de instancias utilizando solo una fracción del conjunto de datos utilizado por SAM. Con Fast Segment, se logra una velocidad de ejecución 50 veces más rápida que SAM sin comprometer significativamente el rendimiento.'

In [13]:
query = "cuál es la diferencia entre fast sam y mobile sam?"
qa_chain.run(query)

'La diferencia entre FastSAM y MobileSAM se puede resumir en dos aspectos principales: tamaño y velocidad.\n\nEn cuanto al tamaño, MobileSAM es significativamente más pequeño que FastSAM. MobileSAM tiene menos de 10 millones de parámetros, mientras que FastSAM tiene 68 millones de parámetros.\n\nEn cuanto a la velocidad, MobileSAM es más rápido que FastSAM. En una GPU individual, MobileSAM tarda solo 10 ms en procesar una imagen, mientras que FastSAM tarda 40 ms. Esto significa que MobileSAM es 4 veces más rápido que FastSAM.\n\nAdemás de estas diferencias, también se menciona que MobileSAM tiene un rendimiento superior en términos de mIoU (Índice de Jaccard medio) en comparación con FastSAM. Esto sugiere que la predicción de máscaras de MobileSAM es más similar a la del SAM original que la de FastSAM.\n\nEn resumen, MobileSAM es más pequeño, más rápido y tiene un rendimiento superior en comparación con FastSAM.'